In [1]:
#Used in  biomedical applications where apart from image classification we also need to know the region of abnormality
# a U net is capable for classifying every single pixel and the input and output here have the same dims


In [4]:
import tensorflow as tf 
import keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, Input
from keras.models import Model, Sequential

In [11]:
def build_model(start_neurons, input_layer):

  #conv_layer1 -> conv_layer2 -> max_pooling -> dropout(optional) Contracting path architecture

  conv1 = Conv2D(start_neurons * 1, (3, 3), activation = 'relu', padding = 'same')(input_layer)
  conv1 = Conv2D(start_neurons * 1, (3, 3), activation = 'relu', padding = 'same')(conv1)
  pool1 = MaxPooling2D((2, 2))(conv1)
  drop1 = Dropout(0.25)(pool1)

  conv2 = Conv2D(start_neurons * 2, (3, 3), activation = 'relu', padding = 'same')(drop1)
  conv2 = Conv2D(start_neurons * 2, (3, 3), activation = 'relu', padding = 'same')(conv2)
  pool2 = MaxPooling2D((2, 2))(conv2)
  drop2 = Dropout(0.5)(pool2)

  conv3 = Conv2D(start_neurons * 4, (3, 3), activation = 'relu', padding = 'same')(drop2)
  conv3 = Conv2D(start_neurons * 4, (3, 3), activation = 'relu', padding = 'same')(conv3)
  pool3 = MaxPooling2D((2, 2))(conv3)
  drop3 = Dropout(0.5)(pool3)

  conv4 = Conv2D(start_neurons * 8, (3, 3), activation = 'relu', padding = 'same')(drop3)
  conv4 = Conv2D(start_neurons * 8, (3, 3), activation = 'relu', padding = 'same')(conv4)
  pool4 = MaxPooling2D((2, 2))(conv4)
  drop4 = Dropout(0.5)(pool4)


  #Middle layer of the UNet, no MP and Dropout
  conv5_m = Conv2D(start_neurons * 16, (3, 3), activation = 'relu', padding = 'same')(drop4)
  conv5_m = Conv2D(start_neurons * 16, (3, 3), activation = 'relu', padding = 'same')(conv5_m)

  #Expansive path of the network - conv_2d_transpose -> concatenate -> conv_layer1 -> conv_layer2
  tconv_4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides = (2, 2), padding = 'same')(conv5_m)
  concat1 = concatenate([tconv_4, conv4]) #the channels dims are restored after concat (halved by transposed conv)
  uconv4 = Dropout(0.5)(concat1)
  uconv4 = Conv2D(start_neurons * 8, (3, 3), activation='relu', padding = 'same')(uconv4)
  uconv4 = Conv2D(start_neurons * 8, (3, 3), activation='relu', padding = 'same')(uconv4)

  tconv_3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides = (2, 2), padding = 'same')(uconv4)
  concat2 = concatenate([tconv_3, conv3]) #the channels dims are restored after concat (halved by transposed conv)
  uconv3 = Dropout(0.5)(concat2)
  uconv3 = Conv2D(start_neurons * 4, (3, 3), activation='relu', padding = 'same')(uconv3)
  uconv3 = Conv2D(start_neurons * 4, (3, 3), activation='relu', padding = 'same')(uconv3)

  tconv_2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides = (2, 2), padding = 'same')(uconv3)
  concat3 = concatenate([tconv_2, conv2]) #the channels dims are restored after concat (halved by transposed conv)
  uconv2 = Dropout(0.5)(concat3)
  uconv2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding = 'same')(uconv2)
  uconv2 = Conv2D(start_neurons * 2, (3, 3), activation='relu', padding = 'same')(uconv2)  

  tconv_1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides = (2, 2), padding = 'same')(uconv2)
  concat4 = concatenate([tconv_1, conv1]) #the channels dims are restored after concat (halved by transposed conv)
  uconv1 = Dropout(0.5)(concat4)
  uconv1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding = 'same')(uconv1)
  uconv1 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding = 'same')(uconv1)

  #output layer
  out_layer = Conv2D(1, (1, 1), padding = 'same', activation = 'sigmoid')(uconv1)

In [12]:
from keras.optimizers import Adam

In [ ]:

input_image = Input((img_h, img_w, 1), name = 'image')
model = build_model(start_neurons=16, input_layer = input_image)
model.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])